# For figure TS13

Temperature response to forcing 1750-2100

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
import matplotlib.patches as mp
from netCDF4 import Dataset
import warnings

from ar6.utils.h5 import *

In [2]:
# TODO - sort out figure sizing

pl.rcParams['figure.figsize'] = (18/2.54, 11/2.54)
pl.rcParams['font.size'] = 11
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.size'] = 0
pl.rcParams['xtick.top'] = True
pl.rcParams['figure.dpi'] = 150

In [3]:
results = load_dict_from_hdf5('../data_output_large/twolayer_SSPs.h5')

In [4]:
results.keys()

dict_keys(['ssp119', 'ssp119_climuncert', 'ssp119_remove_aerosol', 'ssp119_remove_aerosol_climuncert', 'ssp119_remove_bc_on_snow', 'ssp119_remove_bc_on_snow_climuncert', 'ssp119_remove_ch4', 'ssp119_remove_ch4_climuncert', 'ssp119_remove_co2', 'ssp119_remove_co2_climuncert', 'ssp119_remove_hfc', 'ssp119_remove_hfc_climuncert', 'ssp119_remove_o3', 'ssp119_remove_o3_climuncert', 'ssp119_remove_other_anthro', 'ssp119_remove_other_anthro_climuncert', 'ssp119_remove_other_ghg', 'ssp119_remove_other_ghg_climuncert', 'ssp119_remove_slcf', 'ssp119_remove_slcf_climuncert', 'ssp126', 'ssp126_climuncert', 'ssp126_remove_aerosol', 'ssp126_remove_aerosol_climuncert', 'ssp126_remove_bc_on_snow', 'ssp126_remove_bc_on_snow_climuncert', 'ssp126_remove_ch4', 'ssp126_remove_ch4_climuncert', 'ssp126_remove_co2', 'ssp126_remove_co2_climuncert', 'ssp126_remove_hfc', 'ssp126_remove_hfc_climuncert', 'ssp126_remove_o3', 'ssp126_remove_o3_climuncert', 'ssp126_remove_other_anthro', 'ssp126_remove_other_anthro_cl

In [5]:
results['ssp245']['surface_temperature'].shape

(351, 2237)

In [6]:
results['ssp245']['surface_temperature'][269].mean()

1.288290462454323

In [7]:
forcings = ['co2', 'other_ghg', 'other_anthro']
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
for scenario in scenarios:
    for forcing in forcings:
        print(scenario, forcing, (results[scenario]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351]).mean())

ssp119 co2 1.184257818138237
ssp119 other_ghg 0.4861647258767829
ssp119 other_anthro -0.25795296675811524
ssp126 co2 1.5144231930207592
ssp126 other_ghg 0.5063625349676361
ssp126 other_anthro -0.23845712442740466
ssp245 co2 2.2183557849766404
ssp245 other_ghg 0.8222079954746065
ssp245 other_anthro -0.2982050143319578
ssp370 co2 2.9466805281762944
ssp370 other_ghg 1.34018733848262
ssp370 other_anthro -0.600848755105281
ssp585 co2 3.598980300937219
ssp585 other_ghg 1.2204077450601272
ssp585 other_anthro -0.35563907643431164


In [8]:
(results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][0]).shape

(20, 2237)

In [9]:
forcings = ['co2', 'other_ghg', 'other_anthro']

AR6_forc = {}
AR6_ecsforc = {}
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

base_periods = {
    '1750': 0,
    '1850-1900': slice(100,151),
    '1995-2014': slice(245,265),
    '2010-2019': slice(260,270)
}

for scenario in scenarios:
    AR6_forc[scenario] = {}
    AR6_ecsforc[scenario] = {}
    for forcing in forcings:
        AR6_forc[scenario][forcing] = {}
        AR6_ecsforc[scenario][forcing] = {}
        for base_period in base_periods:
            AR6_forc[scenario][forcing][base_period] = np.zeros(5)
            AR6_forc[scenario][forcing][base_period] = np.percentile(
                (
                    (results[scenario]['surface_temperature'][331:351].mean(axis=0) - results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                    (results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351].mean(axis=0) - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                ), (5,16,50,84,95)
            )
            AR6_ecsforc[scenario][forcing][base_period] = np.zeros(5)
            AR6_ecsforc[scenario][forcing][base_period] = np.percentile(
                (
                    (results['%s_climuncert' % scenario]['surface_temperature'][331:351].mean(axis=0) - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                    (results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][331:351].mean(axis=0) - results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                ), (5,16,50,84,95)
            )
    AR6_forc[scenario]['anthro'] = {}
    AR6_ecsforc[scenario]['anthro'] = {}
    for base_period in base_periods:
        AR6_forc[scenario]['anthro'][base_period] = np.percentile(results[scenario]['surface_temperature'][331:351].mean(axis=0) - results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))
        AR6_ecsforc[scenario]['anthro'][base_period] = np.percentile(results['%s_climuncert' % scenario]['surface_temperature'][331:351].mean(axis=0) - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))

In [10]:
print(AR6_ecsforc['ssp245']['anthro']['1750'])
print(AR6_ecsforc['ssp245']['co2']['1750'])
print(AR6_ecsforc['ssp245']['other_ghg']['1750'])
print(AR6_ecsforc['ssp245']['other_anthro']['1750'])

[2.06093682 2.29201153 2.75741662 3.32491126 3.77580863]
[1.60798856 1.82667164 2.23003025 2.73381763 3.16305307]
[0.5914765  0.66602244 0.82773111 1.02892951 1.20466559]
[-0.59556644 -0.46164841 -0.29546272 -0.18396343 -0.13168723]


In [11]:
chapter4_tas = {}
varnames = {
    '05': 'Q05',
    'central': 'mean',
    '95': 'Q95'
}
for scenario in scenarios:
    chapter4_tas[scenario] = {}
    for pc in ['05','central','95']:
        nc = Dataset('../data_input/chapter4_assessed_ranges/assessed_%s_%s.nc' % (scenario, pc))
        chapter4_tas[scenario][pc] = nc.variables[varnames[pc]][:]
        #if pc=='95' and scen=='ssp119':
            #print(nc.variables['time'])
            #print(nc.variables['time'][:])
            #print(chapter4_tas[scen][pc])
        nc.close()
    print(scenario, chapter4_tas[scenario]['05'][-1], chapter4_tas[scenario]['central'][-1], chapter4_tas[scenario]['95'][-1])

ssp119 0.24225383730895733 0.5643103232211051 0.9575484813364052
ssp126 0.506950733053811 0.9043159359522661 1.4826491059645592
ssp245 1.243934996453968 1.814827243577203 2.585020146104215


/nfs/b0110/Users/mencsm/miniconda3/envs/ar6/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()


ssp370 1.997464504676968 2.7563576355060757 3.745606165340795
ssp585 2.4421390507522727 3.5006266858546886 4.817334752006395


In [12]:
for scenario in scenarios:
    print(scenario, AR6_ecsforc[scenario]['anthro']['1995-2014'][[0,2,4]])

ssp119 [0.18526482 0.4873317  1.00715859]
ssp126 [0.48410125 0.8561092  1.48209107]
ssp245 [1.24454062 1.8227577  2.71540659]
ssp370 [2.04358786 2.77952582 3.79594328]
ssp585 [2.55150771 3.56435529 5.02484144]


In [13]:
output = []
for scenario in scenarios:
    for base_period in base_periods:
        for forcing in ['co2', 'other_ghg', 'other_anthro', 'anthro']:
            output.append([scenario, forcing, base_period, 
                           AR6_ecsforc[scenario][forcing][base_period][0],
                           AR6_ecsforc[scenario][forcing][base_period][1],
                           AR6_ecsforc[scenario][forcing][base_period][2],
                           AR6_ecsforc[scenario][forcing][base_period][3],
                           AR6_ecsforc[scenario][forcing][base_period][4],
                        ])

    output.append([scenario, 'chapter4', '1995-2014', chapter4_tas[scenario]['05'][-1],
                   np.nan, 
                   chapter4_tas[scenario]['central'][-1],
                   np.nan,
                   chapter4_tas[scenario]['95'][-1]])
df = pd.DataFrame(output, columns=['scenario', 'forcing', 'base_period', 'p05','p16','p50','p84','p95'])
df.to_csv('../data_output/ts_warming_ranges.csv')